In [92]:
def pdhg_torch(c, G, h, A, b, l, u, is_neg_inf, is_pos_inf, l_dual, u_dual, device, max_iter=1000000, tol=1e-4, verbose=True, term_period=1000):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using the Primal-Dual Hybrid Gradient (PDHG) algorithm.

    Args:
      c, G, h, A, b, l, u: torch tensors representing the problem data
      is_pos_inf: torch tensor indicating which elements of u are +inf
      is_neg_inf: torch tensor indicating which elements of l are -inf
      l_dual: torch tensor representing the lower bounds of the dual variables
      u_dual: torch tensor representing the upper bounds of the dual variables
      device: torch device (cpu or cuda)
      max_iter: maximum number of iterations
      tol: tolerance for convergence
      verbose: whether to print termination information
      term_period: period for termination checks

    Returns:
      minimizer, objective value, and number of iterations for convergence
    """
    import torch

    n = c.shape[0]
    m_ineq = G.shape[0] if G.numel() > 0 else 0
    m_eq = A.shape[0] if A.numel() > 0 else 0

    # Combine constraints
    combined_matrix_list = []
    rhs = []
    if m_ineq > 0:
        combined_matrix_list.append(G)
        rhs.append(h)
    if m_eq > 0:
        combined_matrix_list.append(A)
        rhs.append(b)

    if not combined_matrix_list:
        raise ValueError("Both G and A matrices are empty.")

    K = torch.vstack(combined_matrix_list).to(device)           # Combined constraint matrix
    q = torch.vstack(rhs).to(device)                            # Combined right-hand side
    c = c.to(device)

    q_norm = torch.linalg.norm(q, 2)
    c_norm = torch.linalg.norm(c, 2)


    eta = 0.9 / spectral_norm_estimate_torch(K, num_iters=100)
    omega = c_norm / q_norm if c_norm > 0 and q_norm > 0 else 1

    tau = eta / omega
    sigma = eta * omega

    theta = 1.0

    # Initialize primal and dual
    x = torch.zeros((n, 1), device=device)
    x_old = x.clone()
    y = torch.zeros((K.shape[0], 1), device=device)

    #Variables for infeasibility detection
    certificate_flag = None
    certificate_iter = None
    # Tolerances for infeasibility detection
    tol_eq = 1e-2
    tol_feas = 1e-2
    tol_obj = 1e-2
    tol_dual_eq = 1e-2
    tol_cert = 1e-2
    # Previous values for difference calculation
    x_prev_iter = x.clone()
    y_prev_iter = y.clone()
    lam_prev = torch.zeros_like(x)

    for k in range(max_iter):
        x_old.copy_(x)

        # Compute gradient and primal update
        Kt_y = K.T @ y
        grad = c - Kt_y
        x = torch.clamp(x - tau * grad, min=l, max=u)

        #Compute current lambda from projection residual
        # (For infeasibility detection)
        lam = grad
        lam_plus = (-lam).clamp(min=0)
        lam_minus = lam.clamp(min=0)

        # Extrapolate
        x_bar = x + theta * (x - x_old)

        # Dual update
        K_xbar = K @ x_bar
        y += sigma * (q - K_xbar)

        # Project duals:
        if m_ineq > 0:
            y[:m_ineq] = torch.clamp(y[:m_ineq], min=0.0)

        #Infeasibility detection
        if k >= 1:  # Need at least two iterations to compute differences
            # Compute differences from previous iteration
            dx = x - x_prev_iter
            dy = y - y_prev_iter
            dlam = lam - lam_prev
            dlam_minus = dlam.clamp(min=0)
            dlam_plus = (-dlam).clamp(min=0)

            # Check Dual Infeasibility (primal unbounded) certificate using dx
            if m_eq == 0 or (A @ dx).norm() < tol_eq:
                Gdx = G @ dx if m_ineq > 0 else torch.zeros(1, device=device)
                if (m_ineq == 0 or torch.all(Gdx >= -tol_feas)) and c.T @ dx < tol_obj:
                    # Check if dx respects bound constraints (Lambda set)
                    bounds_ok = True
                    for i in range(n):
                        dx_i = dx[i].item()
                        c_i = c[i].item()
                        l_i = l[i].item() if torch.is_tensor(l) else l[i]
                        u_i = u[i].item() if torch.is_tensor(u) else u[i]

                        # Case 1: Both bounds finite -> dx_i must be ~0
                        if l_i > -float('inf') and u_i < float('inf') and abs(dx_i) > tol_feas:
                            bounds_ok = False
                            break
                        # Case 2: Can only increase but c_i >= 0 -> dx_i should not be negative
                        if u_i == float('inf') and c_i >= 0 and dx_i < -tol_feas:
                            bounds_ok = False
                            break
                        # Case 3: Can only decrease but c_i <= 0 -> dx_i should not be positive
                        if l_i == -float('inf') and c_i <= 0 and dx_i > tol_feas:
                            bounds_ok = False
                            break

                    if bounds_ok:
                        certificate_flag = "DUAL_INFEASIBLE"
                        certificate_iter = k
                        if verbose:
                            print(f"Dual infeasibility (primal unbounded) certificate detected at iter {k}")
                        break

            # Check Primal Infeasibility certificate using dy and dlam
            # Split dual variables into inequality and equality parts
                # Check Primal Infeasibility certificate using dy and dlam
                dy_in = dy[:m_ineq] if m_ineq > 0 else torch.zeros(0, device=device)
                dy_eq = dy[m_ineq:] if m_eq > 0 else torch.zeros(0, device=device)

                #Initialize dual_residual with correct shape (n, 1)
                dual_residual = torch.zeros((n, 1), device=device) if n > 0 else torch.zeros(1, device=device)

                if m_ineq > 0:
                    dual_residual += G.T @ dy_in
                if m_eq > 0:
                    dual_residual += A.T @ dy_eq

                #Subtract dlam which has shape (n, 1)
                dual_residual -= dlam

                # Check norm of the entire dual_residual vector
                dual_residual_norm = dual_residual.norm().item()
                dy_in_nonnegative = m_ineq == 0 or torch.all(dy_in >= -tol_feas)

                if dual_residual_norm < tol_dual_eq and dy_in_nonnegative:
                    dual_combo = 0.0

                    # Inequality constraints contribution
                    if m_ineq > 0:
                        dual_combo += (h.T @ dy_in).item()

                    # Equality constraints contribution
                    if m_eq > 0:
                        dual_combo += (b.T @ dy_eq).item()

                    # FIXED: Boundary constraints contribution - handle infinite bounds
                    finite_l_mask = ~torch.isinf(l) & (l != 0)
                    finite_u_mask = ~torch.isinf(u) & (u != 0)

                    if torch.any(finite_l_mask):
                        dual_combo -= (l[finite_l_mask].T @ dlam_minus[finite_l_mask]).item()

                    if torch.any(finite_u_mask):
                        dual_combo -= (u[finite_u_mask].T @ dlam_plus[finite_u_mask]).item()

                    if verbose and k % 1000 == 0:
                        print(f"  l finite count: {finite_l_mask.sum().item()}/{len(l)}")
                        print(f"  u finite count: {finite_u_mask.sum().item()}/{len(u)}")
                        print(f"  dlam_minus finite: {dlam_minus[finite_l_mask].abs().sum().item() if torch.any(finite_l_mask) else 0}")
                        print(f"  dlam_plus finite: {dlam_plus[finite_u_mask].abs().sum().item() if torch.any(finite_u_mask) else 0}")

                    if dual_combo > -tol_cert:
                        certificate_flag = "PRIMAL_INFEASIBLE"
                        certificate_iter = k
                        if verbose:
                            print(f"Primal infeasibility certificate detected at iter {k}")
                        break

        # Update previous iteration values for next difference calculation
        x_prev_iter.copy_(x)
        y_prev_iter.copy_(y)
        lam_prev.copy_(lam)

        # --- Check Termination Every term_period Iterations ---
        if k % term_period == 0:
            # Primal and dual objective
            prim_obj = (c.T @ x)[0][0]
            dual_obj = (q.T @ y)[0][0]

            # Lagrange multipliers from box projection
            lam = project_lambda_box(grad, is_neg_inf, is_pos_inf)
            lam_pos = (l_dual.T @ torch.clamp(lam, min=0.0))[0][0]
            lam_neg = (u_dual.T @ torch.clamp(lam, max=0.0))[0][0]

            adjusted_dual = dual_obj + lam_pos + lam_neg
            duality_gap = abs(adjusted_dual - prim_obj)

            # Primal residual (feasibility)
            residual_eq = A @ x - b if m_eq > 0 else torch.zeros(1, device=device)
            residual_ineq = torch.clamp(h - G @ x, min=0.0) if m_ineq > 0 else torch.zeros(1, device=device)
            primal_residual = torch.norm(torch.vstack([residual_eq, residual_ineq]), p=2).item()

            # Dual residual (change in x)
            dual_residual = torch.norm(grad - lam, p=2).item()

            if verbose:
                print(f"[{k}] Primal Obj: {prim_obj:.4f}, Adjusted Dual Obj: {adjusted_dual:.4f}, "
                      f"Gap: {duality_gap:.2e}, Prim Res: {primal_residual:.2e}, Dual Res: {dual_residual:.2e}")

            # Termination condition
            if (primal_residual <= tol * (1 + q_norm) and
                dual_residual <= tol * (1 + c_norm) and
                duality_gap <= tol * (1 + abs(prim_obj) + abs(adjusted_dual))):
                if verbose:
                    print(f"Converged at iteration {k}")
                break

    #Return certificate information if detected
    if certificate_flag:
        return x, None, certificate_iter, certificate_flag
    else:
        prim_obj = (c.T @ x)[0][0]
        return x, prim_obj.cpu().numpy(), k, "SOLVED"

In [93]:
def project_lambda_box(grad, is_neg_inf, is_pos_inf):
    """
    Projects the gradient onto the normal cone of the feasible region defined by bounds l and u.

    For each i:
      - If l[i] == -inf and u[i] == +inf: projection is 0
      - If l[i] == -inf and u[i] is real: clamp to ≤ 0 (R⁻)
      - If l[i] is real and u[i] == +inf: clamp to ≥ 0 (R⁺)
      - If both are finite: no projection (keep full value)

    Args:
        grad: (n, 1) gradient vector (torch tensor)
        l: (n, 1) lower bounds (torch tensor)
        u: (n, 1) upper bounds (torch tensor)

    Returns:
        projected: (n, 1) projected gradient (interpreted as λ)
    """
    import torch
    projected = torch.zeros_like(grad)

    # Case 1: (-inf, +inf) → {0}
    unconstrained = is_neg_inf & is_pos_inf
    projected[unconstrained] = 0.0

    # Case 2: (-inf, real) → R⁻ → clamp at 0 from above
    neg_only = is_neg_inf & ~is_pos_inf
    projected[neg_only] = torch.clamp(grad[neg_only], max=0.0)

    # Case 3: (real, +inf) → R⁺ → clamp at 0 from below
    pos_only = ~is_neg_inf & is_pos_inf
    projected[pos_only] = torch.clamp(grad[pos_only], min=0.0)

    # Case 4: (real, real) → full space → keep gradient
    fully_bounded = ~is_neg_inf & ~is_pos_inf
    projected[fully_bounded] = grad[fully_bounded]

    return projected


def spectral_norm_estimate_torch(A, num_iters=10):
  """
  Estimates the spectral norm of a matrix A with enough acuracy to use in
  setting the step size of the PDHG algorithm.
  """
  import torch

  b = torch.randn(A.shape[1], 1, device=A.device)
  for _ in range(num_iters):
      b = A.T @ (A @ b)
      b /= torch.norm(b)
  return torch.norm(A @ b)

In [94]:
def pdhg_solver(mps_file_path, max_iter=100000, tol=1e-4, term_period=1000, verbose=True):
    """
    Full PDHG solver implementation using PyTorch.

    Args:
      mps_file_path (str): Path to the MPS file.
      max_iter (int, optional): Maximum number of iterations. Defaults to 10000.
      tol (float, optional): Tolerance for convergence. Defaults to 1e-4. Use 1e-8 for high accuracy
      term_period (int, optional): Period for termination checks. Defaults to 1000.
      verbose (bool, optional): Whether to print termination check information. Defaults to True.

    Returns:
      The minimizer, objective value, and number of iterations for convergence.
    """
    import torch
    import cplex
    from mps_to_standard_form import mps_to_standard_form_torch

    # --- Device Selection ---
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"PyTorch is using ROCm/CUDA device: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device('cpu')
        print("ROCm/CUDA not available. PyTorch is using CPU.")

    # --- Parameter Loading ---
    try:
        c, G, h, A, b, l, u = mps_to_standard_form_torch(mps_file_path, device=device)
    except Exception as e:
        print(f"Failed to load MPS file: {e}")
        exit(1)

    is_neg_inf = torch.isinf(l) & (l < 0)
    is_pos_inf = torch.isinf(u) & (u > 0)

    l_dual = l.clone()
    u_dual = u.clone()

    l_dual[is_neg_inf] = 0
    u_dual[is_pos_inf] = 0

    # --- Run PDHG Solver on the GPU or CPU ---
    minimizer, obj_val, iterations, status = pdhg_torch(
        c, G, h, A, b, l, u, is_neg_inf, is_pos_inf,
        l_dual, u_dual, device=device, max_iter=max_iter,
        tol=tol, verbose=verbose, term_period=term_period
    )

    print(f"Final Status: {status}")
    if obj_val is not None:
        print("Objective Value:", obj_val)
    print("Iterations:", iterations)

    if minimizer is not None:
        print("\nMinimizer (first 10 variables):")
        print(minimizer[:10].cpu().numpy())

    return minimizer, obj_val, iterations, status

In [98]:
pdhg_solver("PDLP-AMD-RIPS/datasets/Netlib/feasible/25fv47.mps")

ROCm/CUDA not available. PyTorch is using CPU.
[0] Primal Obj: -51.5168, Adjusted Dual Obj: 61.0452, Gap: 1.13e+02, Prim Res: 9.40e+02, Dual Res: 2.90e+01
[1000] Primal Obj: -7480.1792, Adjusted Dual Obj: 12902.2822, Gap: 2.04e+04, Prim Res: 7.96e+02, Dual Res: 2.11e+01
[2000] Primal Obj: -2519.1389, Adjusted Dual Obj: 3039.8757, Gap: 5.56e+03, Prim Res: 5.10e+02, Dual Res: 1.45e+01
[3000] Primal Obj: -1101.6200, Adjusted Dual Obj: 8613.7090, Gap: 9.72e+03, Prim Res: 5.04e+02, Dual Res: 1.18e+01
[4000] Primal Obj: 1387.6201, Adjusted Dual Obj: 12805.7920, Gap: 1.14e+04, Prim Res: 5.06e+02, Dual Res: 1.25e+01
[5000] Primal Obj: -863.1807, Adjusted Dual Obj: 7110.8164, Gap: 7.97e+03, Prim Res: 4.50e+02, Dual Res: 1.06e+01
[6000] Primal Obj: -389.8796, Adjusted Dual Obj: 6862.4839, Gap: 7.25e+03, Prim Res: 3.48e+02, Dual Res: 1.13e+01
[7000] Primal Obj: 1035.8563, Adjusted Dual Obj: 7516.6729, Gap: 6.48e+03, Prim Res: 2.93e+02, Dual Res: 1.11e+01
[8000] Primal Obj: 832.3160, Adjusted Dual

(tensor([[160.9503],
         [ 81.7296],
         [ 20.7763],
         ...,
         [  0.0000],
         [  0.0000],
         [  0.0000]]),
 array(5301.9307, dtype=float32),
 99999,
 'SOLVED')

In [66]:
import numpy as np

In [4]:
!cd PDLP-AMD-RIPS
!rm -rf PDLP-AMD-RIPS
!git clone --branch main https://github.com/SimplySnap/PDLP-AMD-RIPS.git

/bin/bash: line 1: cd: PDLP-AMD-RIPS: No such file or directory
Cloning into 'PDLP-AMD-RIPS'...
remote: Enumerating objects: 1009, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1009 (delta 147), reused 129 (delta 118), pack-reused 830 (from 2)
Receiving objects: 100% (1009/1009), 61.91 MiB | 25.26 MiB/s, done.
Resolving deltas: 100% (558/558), done.


In [5]:
!pip install cplex
%run PDLP-AMD-RIPS/Packages/setup.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 9.7 MB/s eta 0:00:00
'/content/PDLP-AMD-RIPS/Packages' has been added to the system path.
You can now import .py files from this directory.
